In [2]:
from modifier import Modifier
from global_settings import sumovsls

def modifier_sumo(interval_list, modifier_list):
        # Get interval [1.25, 2.75]
        # Get the values from the modifier array for the interval ID,[value,...]
        # Resample traffic volume with different granularity
        interval_ids = [[int(num) for num in interval] for interval in interval_list]
        mlist = []
        for interval in interval_ids:
            ids = []
            volume = []
            print(f"Print interval {interval}")
            # Get traffic volume for each id
            # We assume, constant volume per interval
            start = modifier_list[interval[0]-1][1][0]
            end = modifier_list[interval[1]-1][1][0]
            # Create step for traffic volume
            step = 25
            for value in range(start, end, step):
                volume.append(value)

            ids = Modifier.rescale(volume,interval[0],interval[1])
            print(f"volume {volume}")
            for i in range(0,len(ids)-1):
                mlist.append([ids[i],[volume[i]]*90])
            print(f"MOD list {mlist}")
        return mlist

In [10]:
# SUMO vsl
from controller_modifier_scenarios import Controller
import numpy as np
from components_configuration import components

modlist = Controller.get_list_of_modifiers(10)
modlist

In [4]:
import numpy as np
from components_configuration import components
from global_settings import simexSettings,mds

from validator_controller import ValidatorController
from modifier_controller import ModifierController
from simulator_controller import SimulatorController
mod_x,sim_y_list = SimulatorController.simulator(modlist,selected_simulator=components['sumo_simulator_novsl'])

In [3]:
mod_x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [4]:
l = [[2600.0, 2675.714285714286, 2754.285714285714, 2835.714285714286, 2920.0, 3007.1428571428573, 3097.1428571428573, 3190.0, 3285.714285714286, 3384.285714285714, 3485.714285714286, 3590.0, 3697.142857142857, 3807.142857142857, 3920.0, 4035.714285714286, 4154.285714285714, 4275.714285714286, 4400.0]]
l= [[2600.0, 2675.714285714286]]
m = [ [i,[i]*3] for i in l[0]]
m

In [4]:
sim_y_list = [386.6527777777778,
 407.91666666666674,
 423.98611111111103,
 446.8888888888889,
 481.0694444444444,
 532.4027777777778,
 556.4999999999999,
 613.8194444444445,
 701.0416666666665,
 733.1666666666663]

In [1]:
import numpy as np
from components_configuration import components
from global_settings import simexSettings,mds

from validator_controller import ValidatorController
from modifier_controller import ModifierController
from simulator_controller import SimulatorController
intervals_list=[[mds['domain_min_interval'], mds['domain_max_interval']]]
# ModifierA np.array(x) ** 2
# ModifierB x * 2 / 3
mod_outcome = ModifierController.control(intervals_list=intervals_list,selected_modifier=components['modifierA'], do_plot=simexSettings['do_plot'])
mod_x_list = mod_outcome[0]
checked_intervals = mod_outcome[1]
print("MAIN mod outcome", mod_outcome) 
print(f"MODXLIST {mod_x_list} checked intervals {checked_intervals}")

In [2]:
# IMPORT LIBRARIES
import numpy as np

from components_configuration import components
from global_settings import simexSettings,mds

from validator_controller import ValidatorController
from modifier_controller import ModifierController
from simulator_controller import SimulatorController
from logger_utils import Logger

validator_controller = ValidatorController()
logger = Logger()
logger_main_arguments = {}
is_main_func = True
# Initialize interval list for the first iteration
intervals_list=[[2000, 3500]]
#intervals_list=[[2900, 3200]]


while is_main_func:
    
    # Calls Modifier Controller
    mod_outcome = ModifierController.control(intervals_list=intervals_list,selected_modifier=components['modifierA'], do_plot=simexSettings['do_plot'])
    mod_x_list = mod_outcome[0]
    checked_intervals = mod_outcome[1]
    print("MAIN mod outcome", mod_outcome)  

    
    # breaks loop if iterations end by granularity reached
    if not mod_x_list: # FALSE IF ['modifier_data_point'] < mdv['modifier_incremental_unit']:
        logger_main_arguments['log_contex'] = 'overall MAIN stats'
        logger_main_arguments['main_status'] = 'no generated points'
        logger_main_arguments['remaining_unfit_intervals'] = checked_intervals
        logger.log_main(logger_main_arguments)
        break
    
    # Calls Simulator
    mod_x,sim_y_list = SimulatorController.simulate(mod_x_list,selected_simulator=components['sumo_simulator_vsl'])
    print(f"MODX {mod_x} and sim_y_list {sim_y_list}")
    assert len(mod_x) == len(sim_y_list)

    print("MAIN modx", mod_x)  

    # Calls Validator controller
    intervals_list =  validator_controller.validate(mod_x_list=np.array(mod_x),sim_y_list=np.array(sim_y_list), selected_validator=components['validator'],global_interval=[mds["domain_min_interval"], mds["domain_max_interval"]])
    print("MAIN interval list from VAL:", intervals_list)
    
    # Updates interval_list to new range output from validator controller
    # No more unfit intervals -> write MAIN log
    if not intervals_list:
        is_main_func = False
        logger_main_arguments['log_contex'] = 'overall MAIN stats'
        logger_main_arguments['main_status'] = 'no unfit intervals'
        logger.log_main(logger_main_arguments)

# MAIN cycle completed/interrupted -> write OVERALL statistics        
logger_main_arguments['log_contex'] = 'Overall Stats'                          
logger_main_arguments['main_status'] = 'end cycle'
logger.log_main(logger_main_arguments)

In [ ]:
 mod_ticks = np.arange(mds["domain_min_interval"], mds["domain_max_interval"], mds["modifier_data_point"])


In [ ]:
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

save_object(validator_controller,"vc_vsl-test.pkl")

In [12]:
import matplotlib.pyplot as plt
from global_settings import vfs
import pickle

plt.figure(figsize=(10, 6))
#vsl = validator_controller_vsl
#nvsl = validator_controller_novsl

# How to read pickle file from
vsl = pickle.load(open("vc_vsl.pkl", "rb"))
nvsl = pickle.load(open("vc_novsl.pkl", "rb"))

# Plot for vsl
#for controller in [validator_controller_vsl,validator_controller_novsl]:
# controller= validator_controller_novsl
#    print(f"X values and Y values {controller.x_values}{controller.y_values} predicted values {controller.predicted_values}")
#    plt.scatter(controller.x_values, controller.y_values, label=f'Original Data ')
#    plt.scatter(controller.x_values, controller.predicted_values, label=f'Predicted y Data ', marker='x')
#    plt.plot(controller.fitted_curve[2], controller.fitted_curve[1], color='red', label=f'Polynomial Regression')
#    plt.plot(controller.fitted_curve[2], controller.fitted_curve[1] + vfs["threshold_y_fitting"], color='black', label=f'threshold ')
#    plt.plot(controller.fitted_curve[2], controller.fitted_curve[1] - vfs["threshold_y_fitting"], color='black', label=f'threshold ')

plt.scatter(vsl.x_values, vsl.y_values, label=f'Original Data ')
plt.scatter(vsl.x_values, vsl.predicted_values, label=f'Predicted y Data ', marker='x')
plt.plot(vsl.fitted_curve[2], vsl.fitted_curve[1], color='red', label=f'Polynomial Regression')
plt.plot(vsl.fitted_curve[2], vsl.fitted_curve[1] + vfs["threshold_y_fitting"], color='black', label=f'threshold ')
plt.plot(vsl.fitted_curve[2], vsl.fitted_curve[1] - vfs["threshold_y_fitting"], color='black', label=f'threshold ')

plt.scatter(nvsl.x_values, nvsl.y_values, label=f'Original Data ')
plt.scatter(nvsl.x_values, nvsl.predicted_values, label=f'Predicted y Data ', marker='x')
plt.plot(nvsl.fitted_curve[2], nvsl.fitted_curve[1], color='red', label=f'Polynomial Regression')
plt.plot(nvsl.fitted_curve[2], nvsl.fitted_curve[1] + vfs["threshold_y_fitting"], color='black', label=f'threshold ')
plt.plot(nvsl.fitted_curve[2], nvsl.fitted_curve[1] - vfs["threshold_y_fitting"], color='black', label=f'threshold ')

plt.xlabel('X Values')
plt.ylabel('Y Values')
plt.title('Fitted Curves of vsl and novsl')
plt.legend()
plt.show()

In [1]:
from components_configuration import components
from modifier_controller import ModifierController
from global_settings import simexSettings,mds

# Initialize interval list for the first iteration
intervals_list=[[mds['domain_min_interval'], mds['domain_max_interval']]]

mod_outcome = ModifierController.control(intervals_list=intervals_list,selected_modifier=components['modifier'], do_plot=simexSettings['do_plot'])
mod_x_list = mod_outcome[0]
checked_intervals = mod_outcome[1]
print("MAIN mod outcome", mod_outcome)

# Static tick does not work?